# Car Advertisement Analysis

This project is intended to process, analyze and visualize the data taken from vehicles_us.csv. The data takes car advertisement listings from 2018 and contains information for 51,525 different vehicle sale listings. The vehicles range in age by model year from 1908 to 2019. The purpose of the data visualization below is to examine how price is influenced by various other factors such as model year, odometer reading, vehicle type and others. 

In [1]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

## Importing the data

In [2]:
df = pd.read_csv('vehicles_us.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         51525 non-null  int64  
 1   model_year    47906 non-null  float64
 2   model         51525 non-null  object 
 3   condition     51525 non-null  object 
 4   cylinders     46265 non-null  float64
 5   fuel          51525 non-null  object 
 6   odometer      43633 non-null  float64
 7   transmission  51525 non-null  object 
 8   type          51525 non-null  object 
 9   paint_color   42258 non-null  object 
 10  is_4wd        25572 non-null  float64
 11  date_posted   51525 non-null  object 
 12  days_listed   51525 non-null  int64  
dtypes: float64(4), int64(2), object(7)
memory usage: 5.1+ MB


The info() method demonstrates that there are 13 columns with up to 51,525 values in each. Some of the values are NaN, which can be representative of a binary figure. For example, the null values in 'is_4wd' simply indicate that the listing is for a vehicle that does not have 4wd. Other missing values within different columns do not make up a large portion of the data and should not have a significant impact. 

In [4]:
df.head()

,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed
0,9400,2011.0,bmw x5,good,6.0,gas,145000.0,automatic,SUV,NaN,1.0,2018-06-23,19
1,25500,NaN,ford f-150,good,6.0,gas,88705.0,automatic,pickup,white,1.0,2018-10-19,50
2,5500,2013.0,hyundai sonata,like new,4.0,gas,110000.0,automatic,sedan,red,NaN,2019-02-07,79
3,1500,2003.0,ford f-150,fair,8.0,gas,NaN,automatic,pickup,NaN,NaN,2019-03-22,9
4,14900,2017.0,chrysler 200,excellent,4.0,gas,80903.0,automatic,sedan,black,NaN,2019-04-02,28


It is apparent from the first look at the 'model' column that the vehicle make and model name would be more useful as separate values within the DataFrame. 

## Cleaning up the DataFrame

In [5]:
#separate the make and model strings into new columns
df[['make','model']] = df['model'].str.split(" ", n=1, expand=True)

In [6]:
df.head()

,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed,make
0,9400,2011.0,x5,good,6.0,gas,145000.0,automatic,SUV,NaN,1.0,2018-06-23,19,bmw
1,25500,NaN,f-150,good,6.0,gas,88705.0,automatic,pickup,white,1.0,2018-10-19,50,ford
2,5500,2013.0,sonata,like new,4.0,gas,110000.0,automatic,sedan,red,NaN,2019-02-07,79,hyundai
3,1500,2003.0,f-150,fair,8.0,gas,NaN,automatic,pickup,NaN,NaN,2019-03-22,9,ford
4,14900,2017.0,200,excellent,4.0,gas,80903.0,automatic,sedan,black,NaN,2019-04-02,28,chrysler


In [7]:
#drop the date_posted column as it is unnecessary
df = df.drop(df.columns[11], axis=1)

In [8]:
df.head()

,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,days_listed,make
0,9400,2011.0,x5,good,6.0,gas,145000.0,automatic,SUV,NaN,1.0,19,bmw
1,25500,NaN,f-150,good,6.0,gas,88705.0,automatic,pickup,white,1.0,50,ford
2,5500,2013.0,sonata,like new,4.0,gas,110000.0,automatic,sedan,red,NaN,79,hyundai
3,1500,2003.0,f-150,fair,8.0,gas,NaN,automatic,pickup,NaN,NaN,9,ford
4,14900,2017.0,200,excellent,4.0,gas,80903.0,automatic,sedan,black,NaN,28,chrysler


## Data Prepocessing

In [9]:
#create a variable that defines a list of manufacturer names
make_options = df['make'].unique()

In [10]:
# it will be useful to know the min and max values for the model_year values
df['model_year'].min()

1908.0

In [11]:
df['model_year'].max()

2019.0

Let's take a look at how many rows actually have data on vehicles older than 1950

In [12]:
older_than_1950 = (df['model_year'] < 1950).value_counts()
older_than_1950

model_year
False    51519
True         6
Name: count, dtype: int64

With only six rows containing cars older than 1950, it would be safe to narrow the range and drop those values

In [13]:
min_year, max_year = int(df['model_year'].min()), int(df['model_year'].max())

Creating a new column for further analysis of vehicle age

In [14]:
df['age'] = 2019 - df['model_year']
df.sample(5)

,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,days_listed,make,age
48747,13900,2009.0,escalade,good,8.0,gas,158883.0,automatic,SUV,NaN,1.0,56,cadillac,10.0
7516,29400,NaN,suburban,excellent,8.0,gas,40655.0,automatic,SUV,NaN,NaN,36,chevrolet,NaN
30175,9998,2013.0,f-150,excellent,8.0,gas,205000.0,automatic,pickup,black,NaN,31,ford,6.0
3142,3450,1989.0,tacoma,good,4.0,gas,NaN,automatic,pickup,NaN,NaN,51,toyota,30.0
45949,5500,2003.0,f350 super duty,good,NaN,diesel,195000.0,automatic,pickup,white,1.0,44,ford,16.0


Using the most recent column to further segment the age of the vehicles into explicit ranges for better visualization

In [15]:
def age_category(x):
    if x<5: return '<5'
    elif x>=5 and x<10: return '5-10'
    elif x>=10 and x<20: return '10-20'
    else: return '>20'

In [16]:
df['age_category'] = df['age'].apply(age_category)

In [17]:
df.head()

,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,days_listed,make,age,age_category
0,9400,2011.0,x5,good,6.0,gas,145000.0,automatic,SUV,NaN,1.0,19,bmw,8.0,5-10
1,25500,NaN,f-150,good,6.0,gas,88705.0,automatic,pickup,white,1.0,50,ford,NaN,>20
2,5500,2013.0,sonata,like new,4.0,gas,110000.0,automatic,sedan,red,NaN,79,hyundai,6.0,5-10
3,1500,2003.0,f-150,fair,8.0,gas,NaN,automatic,pickup,NaN,NaN,9,ford,16.0,10-20
4,14900,2017.0,200,excellent,4.0,gas,80903.0,automatic,sedan,black,NaN,28,chrysler,2.0,<5


### General Analysis

In [18]:
make_group = df.groupby('make')['model'].size()
make_group

make
acura              236
bmw                267
buick              271
cadillac           322
chevrolet        10611
chrysler           838
dodge             1255
ford             12672
gmc               2378
honda             3485
hyundai           1173
jeep              3281
kia                585
mercedes-benz       41
nissan            3208
ram               3316
subaru            1272
toyota            5445
volkswagen         869
Name: model, dtype: int64

From the table above, we can see the most popular makes of vehicles are ford, chevrolet, toyota.

In [30]:
# visualize the popularity of each model
model_group = df.groupby('model').size().sort_values()
model_group 
model_top_20 = model_group.tail(20)
display(model_top_20)

model
mustang              681
cr-v                 685
tahoe                713
focus                754
grand cherokee       806
altima               813
civic                815
tacoma               827
escape               841
explorer             847
sierra 1500          906
silverado 2500hd     915
accord               964
camry                990
2500                1091
wrangler            1119
silverado           1271
1500                1750
silverado 1500      2171
f-150               2796
dtype: int64

AttributeError: 'NoneType' object has no attribute 'sort_values'

As we can see from the analysis of model popularity, the most frequently listed models come from the ford, chevrolet and jeep models. This follows the previous table's indication of most popular makes. 

### Price Analysis

In [ ]:
#determine the average price by vehicle type
avg_price_type = df.groupby('type')['price'].mean().reset_index() 

In [ ]:
fig, ax = plt.subplots(figsize=(5,3))
sns.barplot(data=avg_price_type, x="type", y="price")
ax.set_xlabel('Type')
ax.set_ylabel('Average vehicle price')
ax.set_title('Average price by vehicle type')
plt.xticks(rotation=90)
plt.show()

The above barplot demonstrates that vehicle type has a significant impact on the average list price of a vehicle in the dataset. Notable vehicle types with high average values are bus, convertible, coupe, offroad, pickup and truck. It makes sense when you consider that these types are split between the most utility and luxury-driven vehicle types. 

In [ ]:
#determine the average price by vehicle manufacturer
avg_price_make = df.groupby('make')['price'].mean().reset_index()

In [ ]:
fig, ax = plt.subplots(figsize=(5,3))
sns.barplot(data=avg_price_make, x="make", y="price")
ax.set_xlabel('Make')
ax.set_ylabel('Average vehicle price')
ax.set_title('Average price by vehicle manufacturer')
plt.xticks(rotation=90)
plt.show()

This barplot allows us to view the average list price by vehicle manufacturer, or 'make' in this instance. We can clearly see that Mercedes-Benz greatly exceeds the prices of the other makes in this dataset. The other manufacturers showing relatively higher prices are truck and luxury brands. This matches the trend we saw in the previous visualization.

In [ ]:
# create a scatterplot to see how well price is correlated to odometer aka vehicle mileage
fig, ax = plt.subplots(figsize=(5,3))
sns.scatterplot(data=df, x="odometer", y="price",)
ax.set_ylim(0, 200000)
ax.set_xlim(0, 600000)
ax.set_xlabel('Odometer Reading')
ax.set_ylabel('List Price')
ax.set_title('Odometer Vs Price')
plt.show()

As we can see from the scatterplot above, the general trend for list price is to decrease with higher vehicle mileage, as we would expect. There are some outliers that do not follow this trend, but the vast majority of the datapoints show what we would expect from vehicle prices as they gain more miles on the odometer. 

## General Conlusions

After processing the data and organizing it in a way that is easy to visualize, we are able to conclude that vehicle price is heavily influenced by type, meaning whether it is a small car, truck or bus. This is shown both in interactive histograms on the plotly express render page, as well as average list price by these 'type' categories with the bar charts in this notebook. Vehicle price has a negative correlation with odometer, meaning on average, higher odometer readings means lower prices for the vehicle.

There are several outliers in the dataset among very high mileage vehicles and ones with significantly higher list prices than the rest of the dataset. This skews the data visualization attempts, so it was necessary to filter the outliers out at times to better represent the data and observe the trends more easily. Being able to filter the interactive visuals is quite helpful in representing the effects of vehicle condition on list price, with 'new', 'like new' and 'good' indicating higher list prices compared to the other lower level condition descriptions. 

These methods of analyzing and visualizing the data are helpful in viewing trends among the dataset. The code and visual aids could easily be modified to analyze similar data sets within the industry and likely in others also. 